## 딥러닝 파이토치 교과서 
### chapter 2  
#### 파이토치 기초 문법

#### 텐서 생성 및 변환

In [2]:
import torch
print(torch.tensor([[1,2],[3,4]])) # 2차원 형태의 텐서 생성 
print(torch.tensor([[1,2],[3,4]], device = "cuda:0")) # gpu에 텐서 생성 
print(torch.tensor([[1,2],[3,4]],dtype = torch.float64)) # dtype을 이용하여 텐서 생성

temp = torch.tensor([[1,2],[3,4]])
print(temp.numpy()) # tensor를 넘파이 배열로 변환 

temp = torch.tensor([[1,2],[3,4]],device = "cuda:0")
print(temp.to("cpu").numpy()) #gpu상의 텐서를 cpu의 텐서로 변환한 후 넘파이 배열로 변환

tensor([[1, 2],
        [3, 4]])
tensor([[1, 2],
        [3, 4]], device='cuda:0')
tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64)
[[1 2]
 [3 4]]
[[1 2]
 [3 4]]


#### 텐서의 인덱스 조작

In [3]:
temp = torch.FloatTensor([1,2,3,4,5,6,7]) #1차원 벡터 생성
print(temp[0],temp[1],temp[-1]) # 인덱스로 접근 
print("--------------------") 
print(temp[2:5],temp[4:-1]) #슬라이스로 접근 

tensor(1.) tensor(2.) tensor(7.)
--------------------
tensor([3., 4., 5.]) tensor([5., 6.])


#### 텐서 연산 및 차원 조작

In [4]:
v = torch.tensor([1,2,3]) 
w = torch.tensor([3,4,6]) 

print(w - v)

tensor([2, 2, 3])


In [5]:
#차원 조작
temp = torch.tensor([
    [1,2],
    [3,4]
])
print(temp.shape)
print("----------------")
print(temp.view(4,1)) # 4x1 로 변환 
print("----------------")
print(temp.view(-1)) # 1차원 벡터로 변형
print("----------------")
print(temp.view(1,-1)) # -1은 (1,?)과 같은 의미로 다른 차원으로부터 해당 값을 유추하겠다는 것 여기서는 (1,4)로 유추
print("----------------")
print(temp.view(-1,1)) # 마찬가지로 (?,1)과 같은 의미로 -1에 해당하는 차원은 유추하여 (4,1)로 변환

torch.Size([2, 2])
----------------
tensor([[1],
        [2],
        [3],
        [4]])
----------------
tensor([1, 2, 3, 4])
----------------
tensor([[1, 2, 3, 4]])
----------------
tensor([[1],
        [2],
        [3],
        [4]])


### 2.2.2 데이터 준비

#### 단순하게 파일을 불러와서 사용하는 경우 _ 실제 사용하는 데이터가 없기 때문에 실행 시 오류  

In [ ]:
import pandas as pd
import torch

data = pd.read_csv('data_path') #예제라 실제 데이터는 존재하지 않음.

#CSV파일의 x칼럼의 값을 넘파이 배열로 받아 Tensor로 바꾸어 준다. 
x = torch.from_numy(data['x'].values).unsqueeze(dim=1).float() 
y = torch.from_numy(data['y'].values).unsqueeze(dim=1).float() 


#### 커스텀 데이터셋을 만들어서 사용하는 경우 

In [9]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader 

class CustomDataset(Dataset):
    def __init__(self,csv_file): # csv_file 파라미터를 통해 데이터셋을 불러온다. 
        self.label = pd.read_csv(csv_file)
        
    def __len__(self): #전체 데이터셋의 size를 반환합니다. 
        return len(self.label)
    
    def __getitem__(self,idx): #전체 x와 y데이터 중에 해당 idx번째의 데이터를 가져옵니다. 
        sample = torch.tensor(self.label.iloc[idx,0:3]).int()
        label = torch.tensor(self.label.iloc[idx,3]).int()
        
        return sample, label
    
tensor_dataset = CustomDataset('C:/Users/CoIn241/Desktop/chan/torchbook_data/chap02/data/car_evaluation.csv')
dataset = DataLoader(tensor_dataset,batch_size=4,shuffle = True) 
dataset

#### 파이토치에서 제공하는 데이터셋을 사용하는 경우 

In [11]:
# MNIST 데이터셋 다운 예제 
import torchvision.transforms as transforms

mnist_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(1.0)) #평균이 0.5 표준편차가 1.0이 되도록 데이터 분포를 조정
])

from torchvision.datasets import MNIST
import requests 

download_root = 'donwload_path' #다운받을 경로 지정 

train_dataset = MNIST(download_root,transform=mnist_transform,train=True,downlaod=True)
valid_dataset = MNIST(download_root,transform=mnist_transform,train=False,downlaod=True)
test_dataset = MNIST(download_root,transform=mnist_transform,train=False,downlaod=True)


### 모델 정의

#### 단순 신경망을 정의하는 방법 

In [14]:
import torch.nn as nn
model = nn.Linear(in_features =1, out_features=1, bias = True) 

#### nn.Module()을 상속하여 정의하는 방법 

In [17]:
class MLP(nn.Module):
    def __init__(self,inputs):
        super(MLP,self).__init__()
        self.layer = Linear(inputs,1)  # 계층 정의 
        self.activation = Sigmoid() # 활성화 함수 정의

    def forward(self,X):
        X = self.layer(X)
        X = self.activation(X)
        return X

#### Sequential 신경망을 정의하는 방법 

In [21]:
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=64,kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=64,out_channels=30,kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2))
        
        self.layer3 = nn.Sequential(
            nn.Linear(in_features=30*5*5,out_features=10,bias=True),
            nn.ReLU(inplace=True)
        )
        
        
    def forward(self,x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.shape[0],-1)
        x = self.layer3(x)
        
        return x

model = MLP() # 모델에 대한 객체 생성

print("Printing children")
print(list(model.children())) #같은 수준의 하위 노드를 반환  :::: Sequential 밑에 layer 인데, layer 수준만 
print("printing Modules")
print(list(model.modules())) # 모델 네트워크의 모든 노드를 반환 :::: Sequential 밑에 layer인데, 모두 

Printing children
[Sequential(
  (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
), Sequential(
  (0): Conv2d(64, 30, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
), Sequential(
  (0): Linear(in_features=750, out_features=10, bias=True)
  (1): ReLU(inplace=True)
)]
printing Modules
[MLP(
  (layer1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(64, 30, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Linear(in_features=750, out_features=10, bias=True)
    (1): ReLU(inp

#### 함수로 신경망을 정의하는 방법 

In [22]:
def MLP(in_features=1,hidden_features=20,out_featrues=1):
    hidden = nn.Linear() #인자 지정해줘야 함 오류 발생
    activation = nn.ReLU()
    output = nn.Linear()
    net = nn.Sequential(hidden, activation, output)
    return net

### 2.2.4 모델의 파라미터 정의 
- 손실함수
- 옵티마이저
- 스케줄러 